In [ ]:
################################################################################
# Download dataset and models from google drive
################################################################################

!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
import os.path

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

dataset_training = drive.CreateFile({'id': '175QZrGu2KyjKITBcePwcyFzcL1zyJWi9'})
dataset_training.GetContentFile("dataset2_training_0.zip")

dataset_validation = drive.CreateFile({'id': '1Hu4bm92fmnNNPlvweKUgdaPRkL_uN74Z'})
dataset_validation.GetContentFile("dataset2_validation_0.zip")

model = drive.CreateFile({'id': '1_DaND4hvVBxS_is8GCZS-JSJLiDTYHRY'})
model.GetContentFile("pspnet.h5")

model = drive.CreateFile({'id': '1YpCm6bho9fNCVgY9Bh8WBaXX5OQlFqI1'})
model.GetContentFile("trained_pspnet.h5")


In [ ]:
################################################################################
# Import libraries
################################################################################

import zipfile
from PIL import Image
import numpy as np
from keras.models import Model, load_model
from keras.callbacks import CSVLogger, Callback
from keras import layers
from keras.backend import tf as ktf, eval, set_value
import time
import random

In [0]:
################################################################################
# Generator and its functions
################################################################################

# returns x, y
def batch_images(index, batch_size, image_paths, imgs, trained_model):
    with imgs.open(image_paths[index]) as img:
        img = Image.open(img).transpose(Image.FLIP_LEFT_RIGHT)
    img = img.resize((473, 473))

    y = predict_segmentation(img, trained_model)
    x = np.array(img.convert("L").convert("RGB")) - np.array([[[128, 128, 128]]])

    return np.expand_dims(x, axis=0), y


# Returns one-hot encoded segmentation object (w x h x 150)
def predict_segmentation(img, trained_model):
    data_mean = np.array([[[123.68, 116.779, 103.939]]])
    pixel_img = np.array(img)
    pixel_img = pixel_img - data_mean
    bgr_img = pixel_img[:, :, ::-1]
    segmented_img = trained_model.predict(np.expand_dims(bgr_img, axis=0))
    return segmented_img


# Yields batches of x and y values
def generator_fn(batch_size, images_path, trained_model, validation=False):
    with zipfile.ZipFile(images_path) as imgs:
        image_paths = imgs.infolist()
        n_images = len(image_paths)
        i = 0
        while True:
            if i + batch_size > n_images:
                i = 0
            if validation and i + batch_size > 256:
                i = 0
            x, y = batch_images(i, batch_size, image_paths, imgs, trained_model)
            i += batch_size
            yield x, y

In [0]:
################################################################################
# Loading, training and saving model
################################################################################

# custom pspnet layer
class Interp(layers.Layer):

    def __init__(self, new_size, **kwargs):
        self.new_size = new_size
        super(Interp, self).__init__(**kwargs)

    def build(self, input_shape):
        super(Interp, self).build(input_shape)

    def call(self, inputs, **kwargs):
        new_height, new_width = self.new_size
        resized = ktf.image.resize_images(inputs, [new_height, new_width],
                                          align_corners=True)
        return resized

    def compute_output_shape(self, input_shape):
        return tuple([None, self.new_size[0], self.new_size[1], input_shape[3]])

    def get_config(self):
        config = super(Interp, self).get_config()
        config['new_size'] = self.new_size
        return config


# Returns pspnet
def load_trained_model(path):
    trained_model = load_model(path, custom_objects={'Interp': Interp})
    trained_model._make_predict_function()
    return trained_model

  
# Class for saving and uploading model & csv logger
class Upload2Drive(Callback):
    def __init__(self, model_name, n_epochs):
        self.model_name = model_name
        self.n_epochs = n_epochs
        self.model_checkpoint = drive.CreateFile({"title": self.model_name + ".h5",
                                                "parents": [{"kind": "drive#childList",
                                                             "id": "1b4yDZuEjuCDuEKybgyHBWX85ovIbjGAX"}]})
        self.model_log = drive.CreateFile({"title": self.model_name + ".csv",
                                          "parents": [{"kind": "drive#childList",
                                                       "id": "1b4yDZuEjuCDuEKybgyHBWX85ovIbjGAX"}]})
  
    def on_epoch_end(self, epoch, logs=None):
        try:
            if gauth.access_token_expired:
                gauth.Refresh()
        except:
            print("refresh failed")
        if True or epoch / self.n_epochs > 0.4:
            try:
                self.model.save(self.model_name + ".h5", overwrite=True)
                self.model_checkpoint.SetContentFile(self.model_name + ".h5")
                self.model_log.SetContentFile(self.model_name + ".csv")
            except:
                print("save failed")

            try:
                self.model_checkpoint.Upload()
                self.model_log.Upload()
            except:
                print("upload checkpoint failed")

# Returns list of callbacks
def callbacks(model_name, n_epochs):
    cb = list()
    cb.append(CSVLogger(model_name + ".csv"))
    cb.append(Upload2Drive(model_name, n_epochs))
    return cb


In [ ]:
################################################################################
# Main
################################################################################

model_name = "pspnet_8"
batch_size = 1
batches_per_epoch = 4672
n_epochs = 10
batches_per_validation = 256

trained_model = load_trained_model("trained_pspnet.h5")
model = load_trained_model("pspnet.h5")

training_data_fn = generator_fn(batch_size, "dataset2_training_0.zip", trained_model)
validation_data_fn = generator_fn(batch_size, "dataset2_validation_0.zip", trained_model, validation=True)

start_time = time.time()
model.fit_generator(training_data_fn,
                    epochs=n_epochs,
                    steps_per_epoch=batches_per_epoch,
                    callbacks=callbacks(model_name, n_epochs),
                    validation_data=validation_data_fn,
                    validation_steps=batches_per_validation,
                    verbose=2,
                    max_queue_size=30)

print("Training took: " + str(time.time() - start_time))

!ls